# STAT 201 Project Proposal

## Introduction

   In Canada, cancer is currently the leading cause of death annually with 2 in 5 people expected to be diagnosed once in their lifetime (Canadian Cancer Statistics, 2021). With that, gliomas are the most common malignant tumors in adults (Carrano, 2021). However, there exists certain genetic mutations that can increase the risk of cancer and enhance the progression of cancer in individuals such as the epidermal growth factor receptor (EGFR) (Xu, 2017). While EGFR mutations are known to increase risk of cancer, it is more commonly associated with lung cancer, therefore, it can be useful for clinical treatments and therapies to investigate whether a mutation in the EGFR gene impacts the severity of glioma tumors (Liu, 2020). Additionally, it has been reported that there are greater rates of incidence in men compared to women and the treatment of gliomas requires different approaches between men and women (Carrano, 2021). Hence, it is important to investigate whether there is a difference in the mean age of diagnosis between men and women to better tune screening ages accordingly. 
Thus, our two research questions are as follows: 

1) Do the proportion of patients with the most severe form of glioma, glioblastoma (GBM), differ in those with a EGFR mutation than those without?

2) Is there a significant difference in the average age of diagnosis for gliomas between men and women? 

The dataset we will be using is the Glioma Grading Clinical and Mutation Features Dataset from the UCI Machine Learning Repository. The dataset contains 24 attributes however for our analysis we are only concerned with 4 of them. 

For the first question we will be looking at Grade and EGFR:
    8) EGFR: Gene responsible for the production of the epidermal growth factor receptor (EGFR) (0 = not mutated, 1 = mutated)
    24) Grade: Glioma class information (1 = glioblastoma (GBM), 0 = low grade glioma (LGG))

For the second question we will be looking at Gender and Age_at_diagnosis:
    1) Gender: (0 = male, 1 = female)
    2) Age_at_diagnosis: age of individual at time of diagnosis in years


## Preliminary Results

## Methods

## References

Carrano, Anna, et al. “Sex-Specific Differences in Glioblastoma.” Cells, vol. 10, no. 7, 2021, p. 1783, https://doi.org/10.3390/cells10071783. 

Liu, Haijing, et al. “Spectrum of EGFR Aberrations and Potential Clinical Implications: Insights from Integrative Pan‐cancer Analysis.” Cancer Communications, vol. 40, no. 1, 2020, pp. 43–59, https://doi.org/10.1002/cac2.12005. 

“Release Notice - Canadian Cancer Statistics 2021.” Health Promotion and Chronic Disease Prevention in Canada, vol. 41, no. 11, 2021, pp. 399–399, https://doi.org/10.24095/hpcdp.41.11.09. 

Xu, Hongsheng, et al. “Epidermal Growth Factor Receptor in Glioblastoma.” Oncology Letters, vol. 14, no. 1, 2017, pp. 512–516, https://doi.org/10.3892/ol.2017.6221. 
